# Case Complexity Analysis

This notebook will demonstrate:

1. How to call the ChohoCloud API to generate case complexity analysis result.

To use this notebook, you need to prepare the following for a case:

1. Upper and lower jaw oral scan meshes.

In [33]:
import os
import time
import requests
import json
import trimesh
import urllib
import numpy as np

## Define call rules
Please modify the following code blocks based on the information you obtained from us.

In [34]:
# Chohotech service request URL, sent with the API documentation.
base_url = "<service request URL>"

# Chohotech file service URL, sent with the API documentation.
file_server_url = "<service file server URL>"

# The authentication header must be passed in. Please keep the TOKEN confidential!!! If it is leaked, please contact us immediately to reset it. All tasks using this TOKEN will be charged to your account.
zh_token = "<your company's service Token, sent with the contact>" # All API calls must be authenticated with the token.

user_group = "APIClient" # User group, usually named APIClient.

# Your company's user_id, sent with the API documentation.
user_id = "<your company's user_id>"

# If you have received creds.json, it will be read directly below.
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

loaded creds from creds.json


In [40]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # Must specify postfix, i.e., file extension
                        headers={"X-ZH-TOKEN": zh_token}) # Get signed upload URL
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # Returns a single string JSON "string", can also use json.loads(resp.text)

    resp = requests.put(upload_url, data) # No auth header is needed for uploading to the cloud storage service

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    print(response.text)
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    print(response.text)
    return response.json()

def retrieve_data(urn):
    return requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content

def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

##  Case Complexity Analysis

The API calling pipeline in this chapter can be replaced by following single workflow.

```python
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "case-complexity-analysis", 
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id
}
```

### Acquire teeth segmentation

In [36]:
# upper teeth
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-denoise-prod",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "mesh": {"type":"stl", "data": upload_file("upper.stl")},
      "jaw_type": "Upper"
  },
  'output_config': {
    "teeth_comp": {"type": "ply"}
  }
}
result_upper_jaw = run_job_and_get_results(json_call, 300)


{"run_id":"wf_1759145269-fcf45725-ab3c-4546-8e96-5399b6fb607f"}
workflow id is wf_1759145269-fcf45725-ab3c-4546-8e96-5399b6fb607f
API finished in 98.50284337997437s


In [37]:

# lower teeth
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-denoise-prod",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "mesh": {"type":"stl", "data": upload_file("lower.stl")},
      "jaw_type": "Lower"
  },
  'output_config': {
    "teeth_comp": {"type": "ply"}
  }
}
result_lower_jaw = run_job_and_get_results(json_call, 300)



{"run_id":"wf_1759145384-a994ad5d-6588-4fc0-9fc3-039d6dd3aa50"}
workflow id is wf_1759145384-a994ad5d-6588-4fc0-9fc3-039d6dd3aa50
API finished in 125.42753839492798s


### Acquire target position result

In [38]:

json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "auto-arrange", 
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data":{
      "upper_teeth_dict": result_upper_jaw["teeth_comp"],
      "upper_axis_matrix_dict": result_upper_jaw["axis"],
      "lower_teeth_dict": result_lower_jaw["teeth_comp"],
      "lower_axis_matrix_dict": result_lower_jaw["axis"],

  },

}
# result_upper_jaw["teeth_comp"]
# result_upper_jaw["axis"]
# #result_lower_jaw["teeth_comp"]
# #result_lower_jaw["axis"]
result_arrangement = run_job_and_get_results(json_call, 500)


{"run_id":"sa_service_1759145511-1f00b637-1449-4008-971c-01ce2e0ea304"}
workflow id is sa_service_1759145511-1f00b637-1449-4008-971c-01ce2e0ea304
API finished in 6.9486083984375s


### Case Complexity Analysis

In [43]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "case-complexity-analysis", 
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data":{
      "teeth_dict": {**result_upper_jaw["teeth_comp"], **result_lower_jaw["teeth_comp"]},
      "axis_dict": {**result_upper_jaw["axis"], **result_lower_jaw["axis"]},
      "transformation_dict": result_arrangement["result"]["transformation_dict"],
      "landmarks_dict": {**result_upper_jaw["landmarks"], **result_lower_jaw["landmarks"]},
      "target_out_form": "" # No function for now, pass empty string
  },

}
result_case_complexity= run_job_and_get_results(json_call, 500)

{"run_id":"sa_service_1759146870-5ad3b122-d355-4858-a68c-62a5b77986b1"}
workflow id is sa_service_1759146870-5ad3b122-d355-4858-a68c-62a5b77986b1
API finished in 15.303061962127686s
{"result":{"ipr_per_contact":["UR2-UR1 0.375","UR3-UR2 0.375","UR4-UR3 0.375","UR5-UR4 0.25","UL2-UL1 0.1","UL3-UL2 0.1","UL4-UL3 0.125","UL5-UL4 0.125","LL2-LL1 0.1","LL3-LL2 0.1","LL4-LL3 0.125","LL5-LL4 0.1","LR2-LR1 0.4375","LR3-LR2 0.4375","LR4-LR3 0.4375","LR5-LR4 0.3125","UL1-UR1 0.3125","LR1-LL1 0.375"],"left_class_3_discrepancy":0.0,"lower_spacing_per_arch":5.661785715936972,"upper_midline_deviation":0.3168951904613443,"left_class_2_discrepancy":8.571096176724444,"upper_spacing_per_arch":15.598003892486318,"lower_crowding_per_arch":5.661785715936972,"trusion_per_tooth":{"11":-2.2386907253907773,"12":-2.594805859190173,"13":-0.9628071950067352,"14":1.6749593509001621,"15":0.9470097753967816,"16":1.2129114934857699,"17":-0.09909472889225515,"21":-1.8242978034849084,"22":-2.94440921497734,"23":-0.7599

In [44]:
print("=== Case Complexity Analysis Results ===")
print(json.dumps(result_case_complexity, indent=4, ensure_ascii=False))

=== Case Complexity Analysis Results ===
{
    "result": {
        "ipr_per_contact": [
            "UR2-UR1 0.375",
            "UR3-UR2 0.375",
            "UR4-UR3 0.375",
            "UR5-UR4 0.25",
            "UL2-UL1 0.1",
            "UL3-UL2 0.1",
            "UL4-UL3 0.125",
            "UL5-UL4 0.125",
            "LL2-LL1 0.1",
            "LL3-LL2 0.1",
            "LL4-LL3 0.125",
            "LL5-LL4 0.1",
            "LR2-LR1 0.4375",
            "LR3-LR2 0.4375",
            "LR4-LR3 0.4375",
            "LR5-LR4 0.3125",
            "UL1-UR1 0.3125",
            "LR1-LL1 0.375"
        ],
        "left_class_3_discrepancy": 0.0,
        "lower_spacing_per_arch": 5.661785715936972,
        "upper_midline_deviation": 0.3168951904613443,
        "left_class_2_discrepancy": 8.571096176724444,
        "upper_spacing_per_arch": 15.598003892486318,
        "lower_crowding_per_arch": 5.661785715936972,
        "trusion_per_tooth": {
            "11": -2.2386907253907773,
    